# **CLASIFICADOR DE TEXTO**
En este Colab entrené varios modelos para clasificar las respuestas a las encuestas.

Los datos de entrenamiento son las respuestas a las encuestas anteriores etiquetadas con tres categorías: producto, servicio y general. Agregué algunas respuestas generadas por ChatGPT para aumentar la cantidad de datos en las categorías de servicio y general. Aún así, la mayoría de las respuestas tienen la categoría de producto.

Usando los datos de entrenamiento, ajusté el modelo de lenguaje previamente entrenado con textos españoles [BETO](https://github.com/dccuchile/beto).

Para entrenar el modelo, usé como referencia [estas instrucciones de HuggingFace](https://huggingface.co/docs/transformers/tasks/sequence_classification) y [este Colab](https://github.com/CSCI-3349-F22/lab8/tree/main).

## Parte 1: Autorización e instalación de bibliotecas

In [ ]:
# Primero que nada, cambiá el runtime type a GPU. Esto hace que ande mucho más rápido el programa.
# Runtime –> Change runtime type –> T4 GPU

# Google Colab pone límites a la cantidad de uso que le podés dar a la GPU.
# Si alcanzás el límite, podés usar la CPU. Normalmente, el límite se elimina dentro de dos o tres días.

In [ ]:
# Monta tu unidad de Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Autentica al usuario en Google Colab y configura las credenciales para acceder a Google Sheets.
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Instala las librerías necesarias.
# Este paso puede tardar un poco.
!pip install transformers datasets torch scikit-learn accelerate

## Parte 2: Preparación y tokenización

In [ ]:
# Abre y lee el Sheet que tiene los datos de entrenamiento.
# Este Sheet tiene dos columnas: "respuestas" y "categorías".
f = gc.open("datos_de_entrenamiento").sheet1
all = f.get_all_values()

In [ ]:
# Ahora tenemos una lista de listas.
# Cada lista interna es una fila del Sheet con el formato [respuesta, categoría].

# Imprime la lista y la cantidad de elementos.
print(all)
print(len(all))

In [ ]:
# Inicializa un diccionario para mapear etiquetas a IDs.
label2id = {}
# Y uno más para mapear IDs a etiquetas.
id2label = {}

class_id = 0

# Recorre todas las filas.
for row in all[1:]:
  # Si la etiqueta no está en el diccionario, agrega un nuevo elemento.
  if row[1] not in label2id:
    label2id[row[1]] = class_id
    id2label[class_id] = row[1]
    class_id += 1

# Recorre todas las filas.
for row in all[1:]:
  # Reemplaza la etiqueta con su ID correspondiente.
  row[1] = label2id[row[1]]

# Renombra las columnas (este paso es necesario para el proceso de entrenamiento).
all[0][0] = "text"
all[0][1] = "label"

In [ ]:
# Imprime los diccionarios y la lista.
print(label2id)
print(id2label)

print(all)

In [ ]:
import pandas as pd

# Convierte la lista en un objeto tipo DataFrame.
df = pd.DataFrame(all[1:], columns=all[0])

# Muestra los primeros datos.
df.head(20)

In [ ]:
from datasets import Dataset

# Convierte el Dataframe en un objeto tipo Dataset.
dataset = Dataset.from_pandas(df)

In [ ]:
# TOKENIZACIÓN

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Carga el tokenizador pre-entrenado de BETO.
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

# Tokeniza las respuestas.
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

## Parte 3: Entrenamiento y evaluación

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# STRATIFIED SPLIT

# Convierte el Dataset tokenizado en un Dataframe.
#df_tokenized = pd.DataFrame(tokenized_dataset)

# Divide el Dataframe en datos de entrenamiento y datos de prueba.
#train_df, test_df = train_test_split(
#   df_tokenized,
#   test_size=0.2,
#   stratify=df['label'],
#   random_state=42
#)

# Convierte los Dataframes en Datasets.
#train_dataset = Dataset.from_pandas(train_df)
#test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# NORMAL SPLIT

# Divide el Dataset en datos de entrenamiento y datos de prueba.
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
import accelerate

# Carga el clasificador pre-entrenado de BETO.
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=len(label2id))

# Define los argumentos de entrenamiento.
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results2",
    eval_strategy="epoch",
    logging_dir="/content/drive/MyDrive/logs2",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    weight_decay=0.01,
)

In [ ]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import f1_score

# Define la función de evaluación.
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = f1_score(labels, predictions, average='weighted')
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
# Define el entrenador.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# ENTRENAMIENTO

# Entrena el modelo.
# Puede que aparezca un aviso que te pregunte si querés ejecutar código externo. Poné "y" para continuar.

# Este paso puede tardar mucho tiempo si usás la CPU (más de una hora),
# pero va más rápido si usás la GPU (menos de 10 minutos, dependiendo de la cantidad de datos).
trainer.train()

In [ ]:
# EVALUACIÓN

# Evalúa el modelo.
trainer.evaluate()

In [ ]:
# Guarda el modelo para usarlo en otro Colab.
# Antes de guardar un nuevo modelo, asegurate de cambiar el nombre acá para no perder el modelo anterior.
trainer.save_model("/content/drive/MyDrive/clasificador")

In [ ]:
# Guarda el tokenizador para usarlo en otro Colab.
# Antes de guardar un nuevo tokenizador, asegurate de cambiar el nombre acá para no perder el tokenizador anterior.
tokenizer.save_pretrained("/content/drive/MyDrive/clasificador")

## Parte 4: Métricas de entrenamiento y análisis


**Intento #1**

Learning rate: 2e-5

Batch size: 4

Num epochs: 3

Weight decay: 0.01

Normal split

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.829000	0.495714	0.800000	0.800899
2	0.402700	0.604519	0.823077	0.826417
3	0.265800	0.796109	0.846154	0.839683
4	0.067700	0.624356	0.853846	0.853792
```
```
{'eval_loss': 0.6243564486503601,
 'eval_accuracy': 0.8538461538461538,
 'eval_f1': 0.8537923614846691,
 'eval_runtime': 6.1408,
 'eval_samples_per_second': 21.17,
 'eval_steps_per_second': 5.374,
 'epoch': 4.0}
```

**Intento #2**

Learning rate: 2e-5

Batch size: 4

Num epochs: 3

Weight decay: 0.01

Stratified split

Sin datos generados por ChatGPT

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.636400	0.807438	0.783784	0.748369
2	0.364600	0.596321	0.819820	0.806448
3	0.174100	0.661458	0.810811	0.794967
```

```
{'eval_loss': 0.6614578366279602,
 'eval_accuracy': 0.8108108108108109,
 'eval_f1': 0.7949673811742778,
 'eval_runtime': 3.7887,
 'eval_samples_per_second': 29.297,
 'eval_steps_per_second': 7.39,
 'epoch': 3.0}
```
**Intento #3**

Learning rate: 2e-5

Batch size: 4

Num epochs: 3

Weight decay: 0.01

Normal split

Sin datos generados por ChatGPT

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.651000	0.423073	0.864865	0.840319
2	0.364200	0.480779	0.864865	0.865934
3	0.185200	0.571108	0.864865	0.871241
```
```
{'eval_loss': 0.571107804775238,
 'eval_accuracy': 0.8648648648648649,
 'eval_f1': 0.8712413712413712,
 'eval_runtime': 4.2237,
 'eval_samples_per_second': 26.28,
 'eval_steps_per_second': 6.629,
 'epoch': 3.0}
```

**Intento #4**
model_sinprecio2 (me parece que este es el mejor)

Learning rate: 2e-5

Batch size: 4

Num epochs: 3

Weight decay: 0.01

Stratified split

Sin datos de la categoría de "precio"

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.760400	0.485295	0.779412	0.777759
2	0.484600	0.483265	0.875000	0.874873
3	0.132500	0.575767	0.867647	0.867629
```

```
{'eval_loss': 0.5757667422294617,
 'eval_accuracy': 0.8676470588235294,
 'eval_f1': 0.8676288133119252,
 'eval_runtime': 4.5986,
 'eval_samples_per_second': 29.574,
 'eval_steps_per_second': 7.394,
 'epoch': 3.0}
```

**Intento #5**

Learning rate: 2e-5

Batch size: 4

Num epochs: 3

Weight decay: 0.01

Normal split

Más datos generados por ChatGPT

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.771300	0.529133	0.798561	0.776343
2	0.399200	0.632740	0.820144	0.805573
3	0.157600	0.484480	0.848921	0.837652
```

```
{'eval_loss': 0.4844798445701599,
 'eval_accuracy': 0.8489208633093526,
 'eval_f1': 0.8376516645303023,
 'eval_runtime': 4.8769,
 'eval_samples_per_second': 28.502,
 'eval_steps_per_second': 7.177,
 'epoch': 3.0}
```

**Intento #6**

Learning rate: 2e-5

Batch size: 4

Num epochs: 4

Weight decay: 0.01

Normal split

Más datos generados por ChatGPT

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.730800	0.696785	0.805755	0.800822
2	0.467300	0.545005	0.863309	0.861921
3	0.204200	0.621633	0.856115	0.854983
4	0.070200	0.662710	0.863309	0.861774
```

```
{'eval_loss': 0.6627101898193359,
 'eval_accuracy': 0.8633093525179856,
 'eval_f1': 0.86177382204908,
 'eval_runtime': 4.9749,
 'eval_samples_per_second': 27.94,
 'eval_steps_per_second': 7.035,
 'epoch': 4.0}
```

**Intento #7**

Learning rate: 2e-5

Batch size: 4

Num epochs: 4

Weight decay: 0.01

Stratified split

```

Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.755100	0.494268	0.798561	0.790414
2	0.445800	0.662144	0.856115	0.853485
3	0.111500	0.828094	0.827338	0.825752
4	0.037800	0.907003	0.827338	0.826516
```

```
{'eval_loss': 0.907002866268158,
 'eval_accuracy': 0.8273381294964028,
 'eval_f1': 0.8265160047379337,
 'eval_runtime': 5.746,
 'eval_samples_per_second': 24.191,
 'eval_steps_per_second': 6.091,
 'epoch': 4.0}
```

**Intento #8**

Learning rate: 2e-5

Batch size: 4

Num epochs: 2

Weight decay: 0.01

Stratified split

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.768700	0.466155	0.820144	0.816058
2	0.447300	0.491136	0.848921	0.845716
```

```
{'eval_loss': 0.49113568663597107,
 'eval_accuracy': 0.8489208633093526,
 'eval_f1': 0.8457159464424776,
 'eval_runtime': 5.515,
 'eval_samples_per_second': 25.204,
 'eval_steps_per_second': 6.346,
 'epoch': 2.0}
```

**Intento #9**

Igual que el Intento #5

```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.780700	0.562730	0.791367	0.790950
2	0.475300	0.659334	0.812950	0.811216
3	0.196800	0.659759	0.856115	0.853592
```

```
{'eval_loss': 0.6597588062286377,
 'eval_accuracy': 0.8561151079136691,
 'eval_f1': 0.8535918613387037,
 'eval_runtime': 5.5798,
 'eval_samples_per_second': 24.911,
 'eval_steps_per_second': 6.273,
 'epoch': 3.0}
```
*Un training loss bajo y un validation loss alto pueden indicar sobreajuste (overfitting)